<h1>Sentimental analysis/ Task prioritisation<h1>

In [1]:
from dotenv import load_dotenv
import os

In [2]:
load_dotenv()

True

In [3]:
H2O_API_KEY = os.getenv("H2O_API_KEY") 

<h3>Initialise h2o client<h3>

In [4]:
from h2ogpte import H2OGPTE

client = H2OGPTE(
    address='https://h2ogpte.genai.h2o.ai',
    api_key= H2O_API_KEY,
)

Please install the correct version of H2OGPTE with `pip install h2ogpte==1.4.7`.
You can enable strict version checking by passing strict_version_check=True.


In [5]:
# pip install h2ogpte==1.4.7
# rmb to restart after pip install

In [6]:
# Retrieve a collection, assuming 1 collection
collection_id = client.list_recent_collections(0,5)[0].id


<h3>Ingest documents into collection</h>

In [7]:
with open('meeting_minute.txt', 'rb') as f:
    meeting_data2 = client.upload('meetingData2.txt', f)
client.ingest_uploads(collection_id, [meeting_data2])


Job(id='7e092874-f048-4e12-a756-a5ba76bd6b7a', passed=1.0, failed=0.0, progress=1.0, completed=True, canceled=False, date=datetime.datetime(2024, 3, 23, 16, 43, 52, tzinfo=TzInfo(UTC)), kind=<JobKind.IngestUploadsJob: 'IngestUploadsJob'>, statuses=[JobStatus(id='7294d974664446aea89605922b1dca67', status='Collecting done.'), JobStatus(id='16b14b9d57f94c39b4446ff557971f4c', status='Indexing done.')], errors=[], last_update_date=datetime.datetime(2024, 3, 23, 16, 43, 56, tzinfo=TzInfo(UTC)), duration='4s')

<h3>Initialise chat session</h3>

In [8]:
chat_session_id = client.create_chat_session(collection_id)

<h3>Initialise RAG</h3>

In [9]:
# Custom RAG prompting 
def urgency_ordering(client, chat_session_id, llm_index, query):
    llm_list = ['h2oai/h2ogpt-4096-llama2-70b-chat',
                'h2oai/h2ogpt-4096-llama2-13b-chat',
                'h2oai/h2ogpt-32k-codellama-34b-instruct',
                'mistralai/Mixtral-8x7B-Instruct-v0.1',
                'mistral-small-latest',
                'mistral-medium',
                'mistral-large-latest',
                ]
    with client.connect(chat_session_id) as session:
        llm = llm_list[llm_index]
        answer = session.query(
            message=query,
            system_prompt="""You are a very helpful Large Language Model. 
            I will give you a $200 tip if you answer the question correctly, otherwise I will lose my job.'
            """,
            pre_prompt_query="""Pay attention and remember the information below. 
            You will need to use only the given document context to answer the question or imperative at the end.\n
            """,
            llm=llm,
            timeout=60,
        ).content
        print(f"\n\n\n{llm}\n" + answer)

In [12]:
variant1 = urgency_ordering(
    client, 
    chat_session_id,
    0,
    """
    According to only the information in the document sources provided within the collection, retreive the 
    action items and meeting transcript and output all actions with a priority category. 
    Categorise them by either 'Low', 'Medium', or 'High'.
    To determine priority category, score the action items according to the urgency and deadline of the items.
    Importance of an action item depends on the action tasks' deadline, impact to a project, team or organisation,
    dependencies of other action tasks for completion, strategic importance to overall goals and objectives
    of the project or organisation, and complexity and effort.
    For action items that are most important and most urgent that needs to be done as soon as possible, categorise it as 'High'.
    For action items that are important but not as urgent, categorise it as 'Medium'.
    For action items that are not important and not urgent, categorise it as 'Low'.

    Translate all the results to mandarin chinese language for output.
    
    The entire output must be translated to mandarin chinese language and in the following JSON format:
    {"Action": ,
    "Deadline: ,
    "Assigned to": ,
    "Priority": ,
    }

    Translate the keys of the JSON to mandarin chinese language as well.
    """
    )

# variant2 = urgency_ordering(
#     client, 
#     chat_session_id,
#     1,
#     """
#     According to only the information in the document sources provided within the collection, retreive the 
#     action items and meeting transcript and output all actions with a priority category. 
#     Categorise them by either 'Low', 'Medium', or 'High'.
#     To determine priority category, score the action items according to the urgency and deadline of the items.
#     Importance of an action item depends on the action tasks' deadline, impact to a project, team or organisation,
#     dependencies of other action tasks for completion, strategic importance to overall goals and objectives
#     of the project or organisation, and complexity and effort.
#     For action items that are most important and most urgent that needs to be done as soon as possible, categorise it as 'High'.
#     For action items that are important but not as urgent, categorise it as 'Medium'.
#     For action items that are not important and not urgent, categorise it as 'Low'.

#     Output in the following JSON format:
#     {"Action": ,
#     "Deadline: ,
#     "Assigned to": ,
#     "Priority": ,
#     }
#     """
#     )

# variant2 = urgency_ordering(
#     client, 
#     chat_session_id,
#     2,
#     """
#     According to only the information in the document sources provided within the collection, retreive the 
#     action items and meeting transcript and output all actions with a priority category. 
#     Categorise them by either 'Low', 'Medium', or 'High'.
#     To determine priority category, score the action items according to the urgency and deadline of the items.
#     Importance of an action item depends on the action tasks' deadline, impact to a project, team or organisation,
#     dependencies of other action tasks for completion, strategic importance to overall goals and objectives
#     of the project or organisation, and complexity and effort.
#     For action items that are most important and most urgent that needs to be done as soon as possible, categorise it as 'High'.
#     For action items that are important but not as urgent, categorise it as 'Medium'.
#     For action items that are not important and not urgent, categorise it as 'Low'.

#     Output in the following JSON format:
#     {"Action": ,
#     "Deadline: ,
#     "Assigned to": ,
#     "Priority": ,
#     }
#     """
#     )






h2oai/h2ogpt-4096-llama2-70b-chat
Sure, here's the output in the requested format, translated to Mandarin Chinese:

{
"行动": "完成数据收集",
"截止日期": "2024年3月20日",
"分配给": "Alex",
"优先级": "高"
}

{
"行动": "完成数据预处理",
"截止日期": "2024年3月20日",
"分配给": "Alex",
"优先级": "高"
}

{
"行动": "完成代码优化",
"截止日期": "2024年3月27日",
"分配给": "Emily",
"优先级": "高"
}

{
"行动": "完成研究文献综述",
"截止日期": "2024年3月27日",
"分配给": "Chris",
"优先级": "中"
}

{
"行动": "完成展示准备",
"截止日期": "2024年3月31日",
"分配给": "Sam",
"优先级": "高"
}

{
"行动": "协调预算资源",
"截止日期": "2024年3月31日",
"分配给": "Sarah",
"优先级": "中"
}

Note: The priority levels are based on the deadlines and importance of each task, and have been translated to Mandarin Chinese as follows:

* 高 (gāo) - High
* 中 (zhōng) - Medium
* 低 (dī) - Low
